In [1]:
import pandas as pd
import duckdb
import psycopg2

In [ ]:
with open('db_password.txt', 'r') as file:
    db_port = file.readline().strip()
    db_password = file.readline().strip()
    db_name = file.readline().strip()

psql_conn = psycopg2.connect(f"port = '{db_port}' dbname = '{db_name}' user = 'postgres' host = 'localhost' password = '{db_password}'")

table_names = ['item', 'lost_item', 'found_item', 'users', 'locations', 'locates', 'stores', 'returns_or_claims', 'category', 'belongs', 'posts', 'members', 'comments', 'message', 'feedback', 'notifies', 'reward'] 
con = duckdb.connect()

for table_name in table_names:
    query_str = "SELECT * FROM " + table_name
    df = pd.read_sql_query(query_str, psql_conn)
    con.register(table_name, df)

psql_conn.close()

In [ ]:
# function that sees SQL command AS input and output table from database
def query(query):
    result = con.execute(query).fetchall()
    column_names = [desc[0] for desc in con.description]
    return column_names, result

In [ ]:
test_id = 'UB05836965'
columns, data = query(f'''
    explain analyze
    SELECT 
        CASE 
            WHEN m.senderid = {test_id} THEN u_r.username
            ELSE u_s.username
        END AS the_username,
        MAX(m.mgtime) AS last_message_time
    FROM MESSAGE AS m
    JOIN USERS AS u_s ON m.senderid = u_s.userid
    JOIN USERS AS u_r ON m.receiverid = u_r.userid
    WHERE m.senderid = {test_id} OR m.receiverid = {test_id}
    GROUP BY the_username
    ORDER BY last_message_time DESC;
          ''')
old_message = pd.DataFrame(data, columns=columns)
old_message

# cost=42742.06, actual_time=340.212

In [ ]:
'''
CREATE INDEX idx_message_sender
ON MESSAGE (senderid);
CREATE INDEX idx_message_receiver
ON MESSAGE (receiverid);
'''

In [ ]:
test_id = 'UB05836965'
columns, data = query(f'''
    explain analyze
    SELECT 
        CASE 
            WHEN m.senderid = {test_id} THEN u_r.username
            ELSE u_s.username
        END AS the_username,
        MAX(m.mgtime) AS last_message_time
    FROM MESSAGE AS m
    JOIN USERS AS u_s ON m.senderid = u_s.userid
    JOIN USERS AS u_r ON m.receiverid = u_r.userid
    WHERE m.senderid = {test_id} OR m.receiverid = {test_id}
    GROUP BY the_username
    ORDER BY last_message_time DESC;
          ''')
new_message = pd.DataFrame(data, columns=columns)
new_message

# cost=5665.70, actual_time=7.487

In [ ]:
'''
DROP INDEX IF EXISTS idx_message_sender;
DROP INDEX IF EXISTS idx_message_receiver;
'''